# Simple Test of Reciprocity of 3 Sources and 3 Recievers in Groningen

### Step 0

Load packages

In [ ]:
#load all packages
import datetime
import pickle
import copy
import os

from sys import argv
from pathlib import Path

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize


from pyaspect.project import *
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import *
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import *
from pyaspect.specfemio.write import *
from pyaspect.specfemio.write import _write_header
from pyaspect.specfemio.read import *
from pyaspect.specfemio.utils import *


import pyaspect.events.gevents as gevents
import pyaspect.events.gstations as gstations
from pyaspect.events.munge.knmi import correct_station_depths as csd_f
import pyaspect.events.mtensors as mtensors
from obspy.imaging.beachball import beach
from obspy import UTCDateTime
import shapefile as sf

### Step 1 

Extract the ndarray of the subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_in_dir}
!ls data/groningen

### Step 6 

Decompress the ndarray of the sliced, subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# set filename then used it to decompress model
ifqn = f'{data_out_dir}/vsliced_subsmp_smth_nam_2017_vp_vs_rho_Q_model_dx100_dy100_dz100_maxdepth5850_sig250.npz'
vslice_gm3d, other_pars = decompress_gm3d_from_file(ifqn)

print()
print('decompressed gridded model\n:',vslice_gm3d) 
print()
print('other parameters:\n',other_pars)
print()

# WARNING: this will unpack all other_pars, if you overwrite a variable of the samename as val(key), then you 
#          may not notice, and this may cause large headaches.  I use it because I am aware of it.
'''
for key in other_pars:
    locals()[key] = other_pars[key]  #this is more advanced python than I think is reasonable for most 
sig_meters = sig
''';

# another way to get these varibles is just use the accessor functions for the gridmod3d.  We need them later.
xmin = other_pars['xmin']
dx   = other_pars['dx']
nx   = other_pars['nx']
ymin = other_pars['ymin']
dy   = other_pars['dy']
ny   = other_pars['ny']
zmin = other_pars['zmin']
dz   = other_pars['dz']
nz   = other_pars['nz']
sig_meters = other_pars['sig']  # this variable is used later
print('sig_meters:',sig_meters)

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
nam_dims = list(vslice_gm3d.get_npoints())
nam_origin = [0,0,-vslice_gm3d.get_gorigin()[2]]
#nam_origin = list(vslice_gm3d.get_gorigin())
#nam_origin[2] *= -1
nam_origin = tuple(nam_origin)
nam_spacing = list(vslice_gm3d.get_deltas())
nam_spacing[2] *=-1
nam_spacing = tuple(nam_spacing)
print('nam_dims:',nam_dims)
print('nam_origin:',nam_origin)
print('nam_spacing:',nam_spacing)

# Edit the spatial reference
grid.dimensions = np.array(nam_dims) + 1
grid.origin = nam_origin  # The bottom left corner of the data set
grid.spacing = nam_spacing  # These are the cell sizes along each axis
nam_pvalues = vslice_gm3d.getNPArray()[0]
print('pvalues.shape:',nam_pvalues.shape)

# Add the data values to the cell data
grid.cell_arrays["values"] = nam_pvalues.flatten(order="F")  # Flatten the array!

# Now plot the grid!
cmap = plt.cm.jet
#grid.plot(show_edges=True,cmap=cmap)
grid.plot(cmap=cmap,opacity=1.0)


In [ ]:
slices = grid.slice_orthogonal()

#slices.plot(show_edges=True,cmap=cmap)
slices.plot(cmap=cmap)

## create random virtual source (to specfem stations, but using reciprocity -- sources)

In [ ]:
#coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords = vslice_gm3d.getLocalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])


#n_rand_p = 1000

n_rand_p = 3
np.random.seed(n_rand_p) #nothing special about using n_rand_p just want reproducible random

#stay away from the edges of the model for derivatives 
# and to avoid boundary effects
xy_pad = 500 

lrx = np.min(xc) + xy_pad
lry = np.min(yc) + xy_pad
lrz = -3400.0

hrx = np.max(xc) - xy_pad
hry = np.max(yc) - xy_pad
hrz = -2600.0

srx = hrx - lrx
sry = hry - lry
srz = hrz - lrz

r_xyz_list = []
for i in range(n_rand_p):
    rx = lrx + srx*np.random.rand()
    ry = lry + sry*np.random.rand()
    rz = lrz + srz*np.random.rand()
    r_xyz_list.append([rx,ry,rz])
    
r_xyz = np.array(r_xyz_list)
    

#r_xyz = np.vstack(np.meshgrid(rx,ry,rz)).reshape(3,-1).T
print('r_xyz:\n',r_xyz)


In [ ]:
pv_rpoints = pv.wrap(r_xyz)
p = pv.Plotter()
slices = grid.slice_orthogonal()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(slices,cmap=cmap,opacity=1)
p.add_mesh(grid,cmap=cmap,opacity=0.50)
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=1.0)

p.show()

## Make Moment Tensors and CMTSolutionHeaders for each tensor

In [ ]:
# this is the path to the project dir on the cluster
my_proj_dir = '/scratch/seismology/tcullison/test_mesh/FWD_Batch_Src_Test'

magnitude = np.pi
strike = [30,45,90] # just making three to test
dip = [30,30,60]
rake = [330,190,20]

l_mt = []
for i in range(len(strike)):
    l_mt.append(MomentTensor(mw=magnitude,strike=strike[i],dip=dip[i],rake=rake[i]))

assert len(l_mt) == len(r_xyz)

for mt in l_mt:
    print(f'mt:\n{mt}')
    
l_cmt_srcs = []
for i in range(len(r_xyz)):
    cmt_h = CMTSolutionHeader(date=datetime.datetime.now(),
                              ename=f'Event-{str(i).zfill(4)}',
                              tshift=0.0,
                              hdur=0.0,
                              lat_yc=r_xyz[i,1],
                              lon_xc=r_xyz[i,0],
                              depth=-r_xyz[i,2],
                              mt=l_mt[i],
                              eid=i,
                              sid=0)
    l_cmt_srcs.append(cmt_h)
    
print()
for cmt in l_cmt_srcs:
    print(f'cmt:\n{cmt}')

## Make Corresponding "Virtual" Recievers (including cross membors for derivatives) for the CMT's

In [ ]:
m_delta = 50.0 # distance between cross stations for derivatives
assert m_delta < xy_pad #see cells above this is padding
#l_grp_vrecs = make_grouped_half_cross_reciprocal_station_headers_from_cmt_list(l_cmt_srcs,m_delta)
l_grp_vrecs = make_grouped_cross_reciprocal_station_headers_from_cmt_list(l_cmt_srcs,m_delta)

ig = 0
for grp in l_grp_vrecs:
    print(f'***** Group: {ig} *****\n')
    ir = 0
    for gvrec in grp:
        print(f'*** vrec: {ir} ***\n{gvrec}')
        ir += 1
    ig += 1

print(len(flatten_grouped_headers(l_grp_vrecs)))
    

## Plot Virtual Receiver Groups

In [ ]:
all_g_xyz = get_xyz_coords_from_station_list(flatten_grouped_headers(l_grp_vrecs))
all_g_xyz[:,2] *= -1 #pyview z-up positive and oposize sign of standard geophysics 
pv_all_points = pv.wrap(all_g_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.5)
p.add_mesh(slices,cmap=cmap,opacity=1.0)
p.add_mesh(pv_all_points, render_points_as_spheres=True, point_size=5,opacity=1.0)
p.show()

## Get receiver/station coordinates created from a different notebook

In [ ]:
# unpickle the Bounding box (from a different notebook)

#ifqn  = data_out_dir + 'bbox_nvl' + str(int(nvl)) + '_nvb' + str(int(nvb))
#ifqn += '_xsft' + str(xshift) + '_ysft' + str(yshift) + '.pickle'
ifqn = data_out_dir + 'bbox_nvl152_nvb197_xsft4400_ysft19100.pickle'
f = open(ifqn, 'rb')
sgf_bbox = pickle.load(f)
f.close()
print()
print('Unpickled Bounding:\n',sgf_bbox)

In [ ]:
# unpickle the events if needed (again from a different notebook)
ifqn = data_out_dir + 'bbox_groning_events.pickle'
f = open(ifqn, 'rb')
bbox_events = pickle.load(f)
f.close()
print()
print('Unpickled Events:\n',bbox_events)

In [ ]:
#Read moment tensors
mt_in_file  = 'data/groningen/events/event_moments.csv' 
!ls {mt_in_file}
bbox_gf_mts = mtensors(mt_in_file)

# get event catalog of the events (ObsPy catalog)
bbox_event_cat = copy.deepcopy(bbox_events.getIncCatalog())

# This is a bit hokey, but it works. Here we update the
# event time from the moment tensor CSV file with thouse
# from the event catalog
bbox_gf_mts.update_utcdatetime(bbox_event_cat)

'''
#for imt in range(len(bbox_gf_mts)):
#    print("Moment-Tensor %d:/n" %(imt),bbox_gf_mts[imt])
'''

# Create a dictionary that maps moment tensors to events
bbox_emap,bbox_mt_cat,bbox_mts = bbox_gf_mts.get_intersect_map_events_mts(bbox_event_cat)
bbox_e2mt_keys = bbox_emap.keys()

# Print a comparison of events to moment tensors
for key in bbox_e2mt_keys:
    print('UTC: event[%d][Date] = %s' %(key,bbox_mt_cat[key].origins[0].time))
    print('UTC:    MT[%d][Date] = %s' %(key,bbox_emap[key]['Date']))
    print('Mag: event[%d][Date] = %s' %(key,bbox_mt_cat[key].magnitudes[0].mag))
    print('Mag:    MT[%d][Date] = %s' %(key,bbox_emap[key]['ML']))
    print()

#replace moment-tensors with only those that intersect with the events in the BoundingBox
bbox_gf_mts.replace_moment_tensors_from_map(bbox_emap)
    
# add mt_catalog to bbox_events
bbox_events.mergeMomentTensorsCatalog(bbox_mt_cat)
merged_bbox_event_cat = bbox_events.getIncCatalog()
print('bbox_event_cat:\n', bbox_event_cat)
print()
print('merged_bbox_event_cat:\n', merged_bbox_event_cat)
print()
print('bbox_mt_cat:\n', bbox_mt_cat)
print()
print('bbox_mt_df:\n', bbox_gf_mts)

In [ ]:
ifqn = data_out_dir + 'bbox_groning_stations.pickle'

print('Unpickling Station Traces')
f = open(ifqn, 'rb')
bbox_straces = pickle.load(f)
f.close()

print('Stations:\n',type(bbox_straces))

In [ ]:
# read shapefiles
shape_in_files  = 'data/groningen/shapefile/Groningen_field' 

gf_shape = sf.Reader(shape_in_files)
print('Groningen Field shape:',gf_shape)

#get coordinates for the Shape-File
s = gf_shape.shape(0)
shape_xy = np.asarray(s.points)

In [ ]:
# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
#from gnam.events.munge.knmi import correct_station_depths as csd_f
bbox_straces.correct_stations(csd_f)

bbox_bb_diam = 1500  #size of the beachball for plotting. I had to play with this parameter
bbox_cmt_bballs = bbox_gf_mts.get_cmt_beachballs(diam=bbox_bb_diam,fc='black')

bbox_mt_coords = bbox_events.getIncCoords()

#get event and borhole keys used for indexing
ekeys = bbox_straces.getEventKeys()
bkeys = bbox_straces.getBoreholeKeys()

#Plot seuence of events with stations 
#for ie in ekeys:
for i in range(1):
    ie = ekeys[i]
    # coordinates for stations that are in the bounding box
    xy3 = bbox_straces.getIncStationCoords(ie,bkeys[0]) #station code G##3
    xy4 = bbox_straces.getIncStationCoords(ie,bkeys[1]) #station code G##4
    
    # coordinates for stations that are G-stations but outside the bounding box
    ex_xy3 = bbox_straces.getExcStationCoords(ie,bkeys[0]) #station code G##3
    ex_xy4 = bbox_straces.getExcStationCoords(ie,bkeys[1]) #station code G##4
    
    # coordinates for stations that are inside the bounding box but there is no data
    er_xy3 = bbox_straces.getErrStationCoords(ie,bkeys[0]) #station code G##3
    er_xy4 = bbox_straces.getErrStationCoords(ie,bkeys[1]) #station code G##4

    fig, ax = plt.subplots(1,figsize=(8,8))
    fig.gca().set_aspect('equal', adjustable='box')
    
    #Groningen Field Shape-File
    ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
    
    #Bounding Box
    ax.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='green',zorder=1)
    
    #Events (reuse event coordinates from cell further above)
    ax.scatter(bbox_mt_coords[ie,0],bbox_mt_coords[ie,1],s=90,c='red',marker='*',zorder=5)
    beach = bbox_cmt_bballs[ie]  #this creates a plot collection for the beachball points
    beach.set_zorder(3)
    ax.add_collection(beach)
    
    #Included stations
    ax.scatter(xy3[:,0],xy3[:,1],s=50,c='blue',marker='v',zorder=3)
    ax.scatter(xy4[:,0],xy4[:,1],s=100,c='gray',marker='o',zorder=2)
    
    #Excluded stations
    ax.scatter(ex_xy3[:,0],ex_xy3[:,1],s=80,c='lightgray',marker='1',zorder=4)
    ax.scatter(ex_xy4[:,0],ex_xy4[:,1],s=100,c='lightgray',marker='2',zorder=3)
    
    #Stations without data
    ax.scatter(er_xy3[:,0],er_xy3[:,1],s=50,c='yellow',marker='v',zorder=4)
    ax.scatter(er_xy4[:,0],er_xy4[:,1],s=100,c='gray',marker='o',zorder=3)
    
    origin_time = bbox_events[ie].origins[0].time
    mag = bbox_events[ie].magnitudes[0].mag
    title_str = 'Event-%d, Origin Time: %s, Magnitude: %1.2f' %(ie,str(origin_time),mag)
    ax.set_title(title_str)
    plt.show()
    


In [ ]:
fig1, ax1 = plt.subplots(1,figsize=(8,8))
fig1.gca().set_aspect('equal', adjustable='box')
    
#Groningen Field Shape-File
ax1.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)

#Bounding Box
ax1.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='green',zorder=1)

#Events (reuse event coordinates from cell further above)
ax1.scatter(bbox_mt_coords[ie,0],bbox_mt_coords[ie,1],s=90,c='red',marker='*',zorder=5)
    
all_xy4 = np.concatenate((xy4,er_xy4),axis=0)
#all_xy4 = xy4

ax1.scatter(all_xy4[:,0],all_xy4[:,1],s=100,c='gray',marker='o',zorder=2)
ax1.scatter(er_xy4[:,0],er_xy4[:,1],s=100,c='yellow',marker='x',zorder=2)

title_str = 'Event-%d, Origin Time: %s, Magnitude: %1.2f' %(ie,str(origin_time),mag)
ax1.set_title(title_str)
plt.show()

## Make random virtual sources

In [ ]:
coords = vslice_gm3d.getLocalCoordsPointsXY()

x_orig = vslice_gm3d.get_gorigin()[0]
y_orig = vslice_gm3d.get_gorigin()[1]

clip_xy = all_xy4[9:13]
print(clip_xy)

s_xyz = np.zeros((len(clip_xy),3))
s_xyz[:,0] = clip_xy[:,0] - x_orig
s_xyz[:,1] = clip_xy[:,1] - y_orig
s_xyz[:,2] = -200

print(s_xyz)

## Plot virtual sources (red) with virtual receivers (white)

In [ ]:
pv_spoints = pv.wrap(s_xyz)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=8,opacity=1,color='red')
#p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.add_mesh(all_g_xyz, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## Make StationHeaders (real recievers) 

In [ ]:
l_real_recs = []
for i in range(len(s_xyz)):
    
    tr_bname = 'tr'
    new_r = StationHeader(name=tr_bname,
                          network='NL', #FIXME
                          lon_xc=s_xyz[i,0],
                          lat_yc=s_xyz[i,1],
                          depth=-s_xyz[i,2], #specfem z-down is positive
                          elevation=0.0,
                          trid=i)
    l_real_recs.append(new_r)
    
for rec in l_real_recs:
    print(rec)


## Make ForceSolutionHeaders for the above virtual sources (including force-triplets for calculation derivatives)

In [ ]:
l_grp_vsrcs = make_grouped_reciprocal_force_solution_triplet_headers_from_rec_list(l_real_recs)

## Make replicates of each virtual receiver list: one for each force-triplet

In [ ]:
l_grp_vrecs_by_vsrcs = make_replicated_reciprocal_station_headers_from_src_triplet_list(l_grp_vsrcs,
                                                                                          l_grp_vrecs)

## Plot virtual sources (red) and virtual receivers (white) FROM headers

In [ ]:
grp_s_xyz = get_unique_xyz_coords_from_solution_list(flatten_grouped_headers(l_grp_vsrcs))
grp_s_xyz[:,2] *= -1 #pyvista z-up is positive

flat_recs = flatten_grouped_headers(flatten_grouped_headers(l_grp_vrecs_by_vsrcs))
grp_r_xyz = get_unique_xyz_coords_from_station_list(flat_recs)
grp_r_xyz[:,2] *= -1 #pyvista z-up is positive

print(len(grp_s_xyz))
print(len(grp_r_xyz))

pv_spoints = pv.wrap(grp_s_xyz)
pv_rpoints = pv.wrap(grp_r_xyz)

p = pv.Plotter()
p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=8,opacity=1,color='red')
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## Make replicates of each "real" receiver list: for each CMT source

In [ ]:
l_grp_recs_by_srcs = make_replicated_station_headers_from_src_list(l_cmt_srcs,l_real_recs)


for i in range(len(l_cmt_srcs)):
    print(f'***** SRC Records for Source: {i} *****\n')
    for j in range(len(l_real_recs)):
        print(f'*** REC Header for Receiver: {j} ***\n{l_grp_recs_by_srcs[i][j]}')
    

## Plot "real" sources (red) and virtual receivers (white) FROM headers

In [ ]:
grp_s_xyz = get_unique_xyz_coords_from_solution_list(l_cmt_srcs)
grp_s_xyz[:,2] *= -1 #pyvista z-up is positive

flat_recs = flatten_grouped_headers(l_grp_recs_by_srcs) #real!
grp_r_xyz = get_unique_xyz_coords_from_station_list(flat_recs)
grp_r_xyz[:,2] *= -1 #pyvista z-up is positive

print(len(grp_s_xyz))
print(len(grp_r_xyz))

pv_spoints = pv.wrap(grp_s_xyz)
pv_rpoints = pv.wrap(grp_r_xyz)

p = pv.Plotter()
p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=12,opacity=1,color='red')
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=8,opacity=0.5)
p.show()

## Make reciprical RecordHeader

In [ ]:
l_flat_vsrcs = flatten_grouped_headers(l_grp_vsrcs)
l_flat_vrecs = flatten_grouped_headers(flatten_grouped_headers(l_grp_vrecs_by_vsrcs))

vrecord_h = RecordHeader(name='Reciprocal-Record',solutions_h=l_flat_vsrcs,stations_h=l_flat_vrecs)

# save the header to disc
vrec_fqp = os.path.join(data_out_dir,'simple_record_h')
_write_header(vrec_fqp,vrecord_h)

#verify file is there
!ls -l {vrec_fqp}

## Make reciprocal project

In [ ]:
test_proj_name = 'ReciprocalTestProject'
test_proj_root_fqp =  os.path.join(data_out_dir, 'tmp/TestProjects/NewMKProj')
test_parfile_fqp =  os.path.join(data_out_dir, 'Par_file')
test_mesh_fqp = '/scratch/seismology/tcullison/test_mesh/MESH-default_batch_force_src'
test_spec_fqp = '/quanta1/home/tcullison/DevGPU_specfem3d'
test_pyutils_fqp = '/quanta1/home/tcullison/myscripts/python/specfem/pyutils'
test_script_fqp = '/quanta1/home/tcullison/myscripts/specfem'

#copy the reciprocal record
test_proj_record_h = vrecord_h.copy()

make_fwd_project_dir(test_proj_name,
                     test_proj_root_fqp,
                     test_parfile_fqp,
                     test_mesh_fqp,
                     test_spec_fqp,
                     test_pyutils_fqp,
                     test_script_fqp,
                     test_proj_record_h,
                     batch_srcs=False,
                     verbose=True,
                     max_event_rdirs=MAX_SPEC_SRC)
                     #max_event_rdirs=)
        

print()
print('ls:')
!ls {test_proj_root_fqp}
print('ls:')
!ls {test_proj_root_fqp}/*/*

## Make Forward/Real RecordHeader

In [ ]:
l_flat_srcs = l_cmt_srcs #NOTE: we don't need to flatten CMT list because they are not grouped
l_flat_recs = flatten_grouped_headers(l_grp_recs_by_srcs) #Note: only one level of flattening

record_h = RecordHeader(name='Forward-Record',solutions_h=l_flat_srcs,stations_h=l_flat_recs)

# save the header to disc
rec_fqp = os.path.join(data_out_dir,'real_simple_record_h')
_write_header(rec_fqp,record_h)

#verify file is there
!ls -l {rec_fqp}

## Make "real" project

In [ ]:
test_real_proj_name = 'ForwardTestProject'
test_proj_root_fqp =  os.path.join(data_out_dir, 'tmp/TestProjects/NewMKProj')
test_parfile_fqp =  os.path.join(data_out_dir, 'Par_file')
test_mesh_fqp = '/scratch/seismology/tcullison/test_mesh/MESH-default_batch_force_src'
test_spec_fqp = '/quanta1/home/tcullison/DevGPU_specfem3d'
test_pyutils_fqp = '/quanta1/home/tcullison/myscripts/python/specfem/pyutils'
test_script_fqp = '/quanta1/home/tcullison/myscripts/specfem'

#copy the forward/real record
test_real_proj_record_h = record_h.copy()

make_fwd_project_dir(test_real_proj_name,
                     test_proj_root_fqp,
                     test_parfile_fqp,
                     test_mesh_fqp,
                     test_spec_fqp,
                     test_pyutils_fqp,
                     test_script_fqp,
                     test_real_proj_record_h,
                     batch_srcs=False,
                     verbose=True,
                     max_event_rdirs=MAX_SPEC_SRC)
                     #max_event_rdirs=2)


print()
print('ls:')
!ls {test_proj_root_fqp}
print('ls:')
!ls {test_proj_root_fqp}/*/*